In [1]:
import random
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth, association_rules

SEED = 1031
random.seed(SEED)
np.random.seed(SEED)

In [2]:
data = pd.read_csv('3425.csv')
data = data.set_index('SRCID')

In [3]:
target = 'A1'
excludes = ['A4F2_agg']
data = data.loc[:, ~data.columns.isin(excludes)]
data

,Mode,A1,A2,undecided_voter,A3,A4a,A4b,A4c,A4d,A4e,...,F1h,F2a,F2b,F2c,p_gender_sdc,p_age_group_sdc,p_education_sdc,p_state_sdc,StateMap,opinionated
SRCID,,,,,,,,,,,,,,,,,,,,,
2,2,2,1,False,8,2,2,2,2,2,...,3,2,3,2,2,4,3,1,1,False
3,2,2,24,False,7,2,3,2,1,2,...,2,5,4,4,2,4,1,4,4,True
4,2,2,1,False,7,2,2,2,1,2,...,3,2,2,2,2,4,3,5,5,False
6,2,2,3,False,6,3,3,2,2,2,...,1,-98,2,2,2,4,3,4,4,False
7,2,2,1,False,5,2,3,2,2,1,...,1,1,1,1,2,4,3,1,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3581,1,2,1,False,4,1,2,1,1,1,...,2,2,1,2,2,2,1,2,2,True
3583,1,2,3,False,9,1,2,1,1,1,...,1,2,3,2,2,4,3,5,5,True
3585,1,1,-98,True,10,1,3,1,1,1,...,2,2,2,3,1,3,1,3,3,True


In [4]:
cols = [i for i in data.keys() if i not in (target)]
inputs = data[cols]
inputs = pd.get_dummies(inputs, prefix=cols, columns=cols)
inputs

,Mode_1,Mode_2,A2_-99,A2_-98,A2_1,A2_2,A2_3,A2_4,A2_5,A2_6,...,StateMap_1,StateMap_2,StateMap_3,StateMap_4,StateMap_5,StateMap_6,StateMap_7,StateMap_8,opinionated_False,opinionated_True
SRCID,,,,,,,,,,,,,,,,,,,,,
2,0,1,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
6,0,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
7,0,1,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3581,1,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
3583,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3585,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [5]:
cols = [target]
targets = data[cols]
targets = pd.get_dummies(targets, prefix=cols, columns=cols)
targets

,A1_-98,A1_1,A1_2,A1_3,A1_4,A1_5
SRCID,,,,,,
2,0,0,1,0,0,0
3,0,0,1,0,0,0
4,0,0,1,0,0,0
6,0,0,1,0,0,0
7,0,0,1,0,0,0
...,...,...,...,...,...,...
3581,0,0,1,0,0,0
3583,0,0,1,0,0,0
3585,0,1,0,0,0,0


In [6]:
df = targets.join(inputs)
frequent_itemsets = fpgrowth(df, min_support=0.4, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
#rules.loc['A1' in rules['consequents']]
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(E1d_2),(B1b_2),0.950016,0.985626,0.936295,0.985557,0.999930,-0.000065,0.995255
1,(B1b_2),(E1d_2),0.985626,0.950016,0.936295,0.949950,0.999930,-0.000065,0.998680
2,(E1c_2),(B1b_2),0.948383,0.985626,0.934662,0.985532,0.999905,-0.000089,0.993544
3,(B1b_2),(E1c_2),0.985626,0.948383,0.934662,0.948293,0.999905,-0.000089,0.998262
4,(E1d_2),(E1c_2),0.950016,0.948383,0.910160,0.958047,1.010190,0.009181,1.230349
...,...,...,...,...,...,...,...,...,...
1487850,(E1c_2),"(E1d_2, Mode_1, B1c_2, B1b_2, undecided_voter_...",0.948383,0.790265,0.757269,0.798484,1.010401,0.007795,1.040789
1487851,(Mode_1),"(E1d_2, E1c_2, B1c_2, B1b_2, undecided_voter_F...",0.940869,0.808886,0.757269,0.804861,0.995024,-0.003787,0.979374
1487852,(B1c_2),"(E1d_2, E1c_2, Mode_1, B1b_2, undecided_voter_...",0.942829,0.797779,0.757269,0.803188,1.006780,0.005100,1.027485
1487853,(B1b_2),"(E1d_2, E1c_2, Mode_1, B1c_2, undecided_voter_...",0.985626,0.762496,0.757269,0.768313,1.007629,0.005733,1.025107


In [7]:
df = targets.join(inputs)
frequent_itemsets = fpgrowth(df, min_support=0.55, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.985626,(B1b_2)
1,0.950016,(E1d_2)
2,0.948383,(E1c_2)
3,0.947403,(undecided_voter_False)
4,0.942829,(B1c_2)
...,...,...
2523,0.833061,"(Mode_1, E1c_2, B1c_2, B1b_2)"
2524,0.798105,"(E1d_2, E1c_2, Mode_1, B1c_2, B1b_2)"
2525,0.762496,"(E1d_2, E1c_2, Mode_1, B1c_2, undecided_voter_..."
2526,0.789938,"(E1c_2, Mode_1, B1c_2, B1b_2, undecided_voter_..."


In [8]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.61)
t = 'A1_2'
rules.loc[[t in ic for ic in rules['consequents']]]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
75827,(B1b_2),(A1_2),0.985626,0.616139,0.607971,0.616838,1.001135,0.000689,1.001825
75828,(E1d_2),(A1_2),0.950016,0.616139,0.590003,0.621045,1.007964,0.004662,1.012948
75831,(undecided_voter_False),(A1_2),0.947403,0.616139,0.589677,0.622414,1.010185,0.005945,1.016619
75832,(E1c_2),(A1_2),0.948383,0.616139,0.588043,0.620048,1.006346,0.003708,1.010290
75835,(B1c_2),(A1_2),0.942829,0.616139,0.583143,0.618503,1.003838,0.002229,1.006198
75837,(E1e_2),(A1_2),0.927801,0.616139,0.581509,0.626761,1.017240,0.009855,1.028459
75839,(E1b_2),(A1_2),0.906893,0.616139,0.560928,0.618516,1.003858,0.002156,1.006232
75840,(Mode_1),(A1_2),0.940869,0.616139,0.583469,0.620139,1.006493,0.003764,1.010531
75843,"(E1d_2, B1b_2)",(A1_2),0.936295,0.616139,0.581836,0.621424,1.008578,0.004948,1.013960
75845,(E1d_2),"(A1_2, B1b_2)",0.950016,0.607971,0.581836,0.612448,1.007364,0.004253,1.011552
